<a href="https://colab.research.google.com/github/IgorOlenchuk/hakaton_2022_kk/blob/main/%D0%A6%D0%9F_%D1%85%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Устанавливаем бибилиотеку Darts (https://unit8co.github.io/darts/README.html)
pip install darts

In [ ]:
# Импортируем необходимые бибилиотеки
import pandas as pd

from darts import TimeSeries
from darts.models import ExponentialSmoothing, NaiveSeasonal

# Утилиты
from pathlib import Path
from datetime import datetime
import time
from tqdm import tqdm

In [ ]:
# Подключаем облачное хранилище
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Фукнция, которая проходит по файлам внутри указанной директории
def do_work(directory):
    pathlist = Path(directory).glob('*.xlsx')

    for file in tqdm(pathlist):
      print('Обрабатываем файл: {file}'.format(file=file))
      file_time(file)

# Функция создания, обучения и предсказания пропущенных данных
def forcast(data, date, x, n):
    series = TimeSeries.from_dataframe(data, date, x)
    n = int(n)
    train, test = series[:-n], series[-n:]
    model = NaiveSeasonal(K=12)
    model.fit(train)
    forecast = model.predict(len(test))
    print('В колонку {col} добавляем {n} значений'.format(col=x, n=n))
    print(' '.join(forecast.pd_series().astype('str')))
    return pd.concat([train.pd_series(), forecast.pd_series()], axis=0).values

# Функция считывает указанный лист из файла и проходит по всем колонкам листа, передавай их на дальнейшую обработку
def read_data(file, sheet):
    format_datetime = lambda x: datetime.strptime(x, '%ym%m')

    data = pd.read_excel(file, 
                        sheet_name=sheet, 
                        index_col=False, 
                        engine='openpyxl', 
                        parse_dates=[0], 
                        date_parser=format_datetime)

    count_var = 0
    count_forecast = 0
    for col in data.columns:
        count_forecast = data[data[col]=='Forecast'][col].count()
        count_var = data[col].count()
        print('В колонке {col} пропущено {count_forecast} значений из {count_var}'.format(col=col, 
                                                                                          count_forecast=count_forecast, 
                                                                                          count_var=count_var))
        if count_forecast>0:
          data[col] = data[col].replace(['Forecast'], 0)
          data[col] = forcast(data, data.columns[0], col, count_forecast)
    return data

# Функция открывает указанный файл и проходит по всем листам. 
# После обработки данных, функция записывает результат в новый файл      
def file_time(file):    
    xls_file = pd.ExcelFile(file)
    out_file = str(file).replace('input', 'output')
    
    for sheet in xls_file.sheet_names:
        new_data = read_data(file, sheet)
        new_data['Unnamed: 0'] = new_data['Unnamed: 0'].dt.strftime('%ym%m')
        new_data.rename(columns = {'Unnamed: 0':''}, inplace = True)

        if sheet == 'Monthly':
            print('Добавляем лист Месяцы')            
            writer = pd.ExcelWriter(out_file)
            print('Сохраняем файл {f}'.format(f=out_file))
            new_data.to_excel(writer, sheet_name=sheet, index=False)
            writer.save()
        elif sheet == 'Quarterly':
            print('Добавляем лист Кварталы')
            my_file = Path(out_file)
            if my_file.is_file():
              writer = pd.ExcelWriter(out_file, engine='openpyxl', mode='a')
            else:
              writer = pd.ExcelWriter(out_file)
            
            print('Сохраняем файл {f}'.format(f=out_file))
            new_data.to_excel(writer, sheet_name=sheet, index=False)
            writer.save()

In [ ]:
# Путь к исходным файлам
directory = '/content/drive/MyDrive/Colab Notebooks/data/input'

do_work(directory)